In [2]:
%pip install -U transformers
%pip install -U accelerate
%pip install datasets gradio evaluate python-dotenv

In [3]:
! git clone https://github.com/mdeff/fma.git

fatal: destination path 'fma' already exists and is not an empty directory.


In [4]:
%cd /content/fma/data


/content/fma/data


In [6]:
from datasets import load_dataset
import urllib.request
from tqdm.auto import tqdm

def show_download_progress(block_num, block_size, total_size):
    progress_bar.update(block_num * block_size - progress_bar.n)

progress_bar = tqdm(unit='B', unit_scale=True, unit_divisor=1024)

urllib.request.urlretrieve("https://os.unil.cloud.switch.ch/fma/fma_metadata.zip", "fma_metadata.zip", reporthook=show_download_progress)
urllib.request.urlretrieve("https://os.unil.cloud.switch.ch/fma/fma_small.zip", "fma_small.zip", reporthook=show_download_progress)

progress_bar.close()

0.00B [00:00, ?B/s]

KeyboardInterrupt: 

In [ ]:
!unzip -o fma_metadata.zip
# !unzip fma_small.zip
# 156 subdirectories

In [1]:
%cd /content/fma
%pip install -r requirements.txt

/content/fma
  Using cached numpy-1.12.1.zip (4.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached pandas-0.19.2.tar.gz (9.2 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# gtzan

In [54]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for marsyas/gtzan contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/marsyas/gtzan
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [57]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [58]:
gtzan["train"][0]
print(gtzan["train"].features["genre"])
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

ClassLabel(names=['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock'], id=None)


'pop'

In [ ]:
import gradio as gr

def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), id2label_fn(example["genre"])


with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True)

# fma_small dataset

In [2]:
import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display

import utils

In [3]:
%cd /content/fma/

# Directory where mp3 are stored.
AUDIO_DIR = "data/fma_small"

# Load metadata and features.
tracks = utils.load('data/fma_metadata/tracks.csv')
genres = utils.load('data/fma_metadata/genres.csv')
features = utils.load('data/fma_metadata/features.csv')
echonest = utils.load('data/fma_metadata/echonest.csv')

np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, genres.shape, features.shape, echonest.shape

/content/fma


((106574, 52), (163, 4), (106574, 518), (13129, 249))

In [4]:
tracks_small = tracks.loc[tracks[('set', 'subset')] <= 'small', [('track', 'genre_top'), ('track', 'bit_rate')]]
print(tracks_small[:5])
print(tracks_small.shape)
most_common_bitrate = tracks_small[('track', 'bit_rate')].value_counts().idxmax()
print(f"The most common bit rate in tracks_small is: {most_common_bitrate}")
tracks_with_same_bitrate = tracks_small.loc[tracks_small[('track', 'bit_rate')] == most_common_bitrate]
print(tracks_with_same_bitrate[:5])
print(tracks_with_same_bitrate.shape)

# get first 1000
tracks_with_same_bitrate = tracks_with_same_bitrate.iloc[:1000]
tracks_with_same_bitrate.shape

             track         
         genre_top bit_rate
track_id                   
2          Hip-Hop   256000
5          Hip-Hop   256000
10             Pop   192000
140           Folk   128000
141           Folk   128000
(8000, 2)
The most common bit rate in tracks_small is: 320000
               track         
           genre_top bit_rate
track_id                     
6779            Folk   320000
6782            Folk   320000
6783            Folk   320000
7011      Electronic   320000
11198            Pop   320000
(3704, 2)


(1000, 2)

In [5]:
%cd /content/fma/data

/content/fma/data


In [6]:
# create files_to_extract.txt based on tracks_with_same_bitrate
with open("files_to_extract.txt", "w") as f:
    for track_id in tracks_with_same_bitrate.index:
        f.write(f"{utils.get_audio_path('', track_id)}\n")

In [ ]:
%%shell
# only unzip the files we need
cd /content/fma/data
mkdir -p fma_small
cat files_to_extract.txt | while read file; do
    mkdir -p "$(dirname "fma_small/$file")"
    unzip -o fma_small.zip "fma_small/$file" -d ./
done


In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(1):
            output = gr.Audio("../fma_small/000/000010.mp3", label="hi")

demo.launch(debug=True)

In [7]:
def get_abs_path(path):
    return os.path.join("/content/fma", path)
file = utils.get_audio_path(AUDIO_DIR, tracks_with_same_bitrate.index[1])
file = get_abs_path(file)
audio = librosa.load(file, sr=22050)[0]
print(audio.shape[0] / 22050)
ipd.Audio(audio, rate=22050)

30.002721088435376


In [9]:
from datasets import Dataset, Audio, ClassLabel

def get_abs_path(path):
    return os.path.join("/content/fma", path)

audio_paths = [get_abs_path(utils.get_audio_path(AUDIO_DIR, track_id)) for track_id in tracks_with_same_bitrate.index]
print(audio_paths[:5])

audio_dataset = Dataset.from_dict({"audio": audio_paths}).cast_column("audio", Audio(sampling_rate=22050))

# add genre column
genre_categories = genres.title.tolist()
# genre_to_index = {genre: index for index, genre in enumerate(genre_categories)}
# index_to_genre = {index: genre for index, genre in enumerate(genre_categories)}
# genres_idx = tracks_with_same_bitrate[('track', 'genre_top')].tolist()
# genres_idx = [genre_to_index[genre] for genre in genres_idx]
audio_dataset = audio_dataset.add_column("genre", tracks_with_same_bitrate[('track', 'genre_top')].tolist())
audio_dataset = audio_dataset.cast_column("genre", ClassLabel(num_classes=len(genre_categories), names=genre_categories))
audio_dataset = audio_dataset.add_column("file", audio_paths)
audio_dataset

['/content/fma/data/fma_small/006/006779.mp3', '/content/fma/data/fma_small/006/006782.mp3', '/content/fma/data/fma_small/006/006783.mp3', '/content/fma/data/fma_small/007/007011.mp3', '/content/fma/data/fma_small/011/011198.mp3']


Casting the dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'genre', 'file'],
    num_rows: 1000
})

In [10]:
audio_dataset = audio_dataset.train_test_split(seed=42, shuffle=True, test_size=0.1)
audio_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'genre', 'file'],
        num_rows: 900
    })
    test: Dataset({
        features: ['audio', 'genre', 'file'],
        num_rows: 100
    })
})

In [71]:
gtzan["train"][0]
print(gtzan["train"].features["genre"])
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

ClassLabel(names=['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock'], id=None)


'pop'

In [11]:
print(audio_dataset["train"].features["genre"])
id2label_fn = audio_dataset["train"].features["genre"].int2str
id2label_fn(audio_dataset["train"][0]["genre"])

ClassLabel(names=['Avant-Garde', 'International', 'Blues', 'Jazz', 'Classical', 'Novelty', 'Comedy', 'Old-Time / Historic', 'Country', 'Pop', 'Disco', 'Rock', 'Easy Listening', 'Soul-RnB', 'Electronic', 'Sound Effects', 'Folk', 'Soundtrack', 'Funk', 'Spoken', 'Hip-Hop', 'Audio Collage', 'Punk', 'Post-Rock', 'Lo-Fi', 'Field Recordings', 'Metal', 'Noise', 'Psych-Folk', 'Krautrock', 'Jazz: Vocal', 'Experimental', 'Electroacoustic', 'Ambient Electronic', 'Radio Art', 'Loud-Rock', 'Latin America', 'Drone', 'Free-Folk', 'Noise-Rock', 'Psych-Rock', 'Bluegrass', 'Electro-Punk', 'Radio', 'Indie-Rock', 'Industrial', 'No Wave', 'Free-Jazz', 'Experimental Pop', 'French', 'Reggae - Dub', 'Afrobeat', 'Nerdcore', 'Garage', 'Indian', 'New Wave', 'Post-Punk', 'Sludge', 'African', 'Freak-Folk', 'Jazz: Out', 'Progressive', 'Alternative Hip-Hop', 'Death-Metal', 'Middle East', 'Singer-Songwriter', 'Ambient', 'Hardcore', 'Power-Pop', 'Space-Rock', 'Polka', 'Balkan', 'Unclassifiable', 'Europe', 'Americana', 

'Hip-Hop'

In [12]:
import gc

gc.collect()

2144

# Features

In [13]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)
sampling_rate = feature_extractor.sampling_rate
sampling_rate

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


16000

In [14]:
from datasets import Audio

# gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))
# print(gtzan)
# gtzan["train"][0]["audio"]

audio_dataset = audio_dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))
print(audio_dataset)
audio_dataset["train"][0]["audio"]

DatasetDict({
    train: Dataset({
        features: ['audio', 'genre', 'file'],
        num_rows: 900
    })
    test: Dataset({
        features: ['audio', 'genre', 'file'],
        num_rows: 100
    })
})


{'path': '/content/fma/data/fma_small/061/061174.mp3',
 'array': array([ 4.32328306e-09,  4.33802416e-09,  9.04991371e-09, ...,
         8.14797282e-02, -2.12462619e-01,  0.00000000e+00]),
 'sampling_rate': 16000}

In [15]:
import numpy as np

sample = audio_dataset["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: -0.000822, Variance: 0.16


In [16]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: 1.59e-08, Variance: 1.0


In [17]:
max_duration = 30.0

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [18]:
# gtzan_encoded = gtzan.map(
#     preprocess_function,
#     remove_columns=["audio", "file"],
#     batched=True,
#     batch_size=100,
#     num_proc=1,
# )
# gtzan_encoded

fma_small_encoded = audio_dataset.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
fma_small_encoded

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [19]:
# gtzan_encoded = gtzan_encoded.rename_column("genre", "label")
fma_small_encoded = fma_small_encoded.rename_column("genre", "label")

In [20]:
id2label = {
    str(i): id2label_fn(i)
    # for i in range(len(gtzan_encoded["train"].features["label"].names))
    for i in range(len(fma_small_encoded["train"].features["label"].names))
}

label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'Old-Time / Historic'

# Model

In [21]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    # push_to_hub=True,
)

In [24]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [26]:
from transformers import Trainer

# trainer = Trainer(
#     model,
#     training_args,
#     train_dataset=gtzan_encoded["train"],
#     eval_dataset=gtzan_encoded["test"],
#     tokenizer=feature_extractor,
#     compute_metrics=compute_metrics,
# )

trainer = Trainer(
    model,
    training_args,
    train_dataset=fma_small_encoded["train"],
    eval_dataset=fma_small_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,3.368700,3.019023,0.300000
2,2.072200,1.929356,0.350000
3,1.784400,1.805264,0.350000
4,1.539700,1.670127,0.440000
5,1.321800,1.646030,0.420000
6,1.197100,1.703816,0.440000
7,1.023300,1.601562,0.470000
8,0.955400,1.685908,0.440000
9,0.726100,1.655754,0.430000


Epoch,Training Loss,Validation Loss,Accuracy
1,3.368700,3.019023,0.300000
2,2.072200,1.929356,0.350000
3,1.784400,1.805264,0.350000
4,1.539700,1.670127,0.440000
5,1.321800,1.646030,0.420000
6,1.197100,1.703816,0.440000
7,1.023300,1.601562,0.470000
8,0.955400,1.685908,0.440000
9,0.726100,1.655754,0.430000
10,0.738100,1.684462,0.440000


TrainOutput(global_step=1130, training_loss=1.679881921278692, metrics={'train_runtime': 5138.7314, 'train_samples_per_second': 1.751, 'train_steps_per_second': 0.22, 'total_flos': 6.150967816906246e+17, 'train_loss': 1.679881921278692, 'epoch': 10.0})